In [1]:
(require '[clojupyter.misc.helper :as helper])
(run! helper/add-dependencies '[[sicmutils "0.15.0"]])

(ns chp1
    (:refer-clojure :exclude [partial zero? + - * / ref compare])
    (:require [sicmutils.env :refer :all]
              [sicmutils.value :as v]))


nil

- The motion of a ball constrained on a surface can be described by Euler-Lagrange equations. There are coeffcients of terms in the Euler-Lagrange equations, connecting the static shape of the surface with the dynamical behavior of the balls. 
- The dynamical evolution of the balls may be viewed as parallel transport of the ball's velocity vector in the direction of the velocity vector. 
- THis motion by parallel transport of velocity is called _geodesic motion_.
- There are deep connections between
    - the dynamics of particles
    - the geometry of the space that the particles move in
- When we express the Lagrangian and the Lagrange equations in differential geometry language, geometric properties become apparent. 
- In the case of systems with no potential energy the Euler-Lagrange equations are equivalent to the geodesic equations on the configuration manifold. 
- The coefficients of terms in the Lagrange equations are Christoffel coefficients, which define parallel transport on the manifold. 

## Lagrange Equations

In [2]:
;; the Lagrangian for a free particle of mass m and velocity v
;; no potential energy
(defn Lfree
    [mass]
    (fn [[t q v]]
        (* (/ 1 2) mass (square v))))

#'chp1/Lfree

In [3]:
;; coordinate transformation
;; generalized coordinates on the sphere -> Cartesian coordinates
(defn sphere->R3
    [R]
    (fn [[t [theta phi] v]]
        (up (* R (sin theta) (cos phi))   ; x
            (* R (sin theta) (sin phi))   ; y
            (* R (cos theta)))))          ; z

#'chp1/sphere->R3

In [15]:
(defn Lsphere
    [m R]
    (compose (Lfree m) (F->C (sphere->R3 R))))

#'chp1/Lsphere

In [7]:
(clojure.repl/source compose)

(defn compose
  "Arity-preserving version of `clojure.core/comp`.

  The arity of a composition is the arity of the rightmost (that is, first to be
  applied) function term in `fns`."
  [& fns]
  (let [a (arity (last fns))]
    (with-meta (apply comp fns) {:arity a})))


nil

In [8]:
(clojure.repl/source F->C)

(defn F->C [F]
  (fn [[t _ v :as local]]
    (->L-state t
               (F local)
               (+ (((partial 0) F) local)
                  (* (((partial 1) F) local) v)))))


nil

In [10]:
(clojure.repl/source ->L-state)

(defn ->L-state
  "Constructs a Lagrangian state, also knows as a local tuple"
  [t q v & as]
  (apply up t q v as))


nil

In [22]:
(clojure.repl/source partial)

(defn partial
  "A shim. Dispatches to [[d/partial]] when all the arguments are integers; falls
  back to [[clojure.core/partial]] (partial function application) otherwise."
  [& selectors]
  (if (every? integer? selectors)
    (apply d/partial selectors)
    (apply core-partial selectors)))


nil

In [13]:
up

#function[sicmutils.structure/up]

In [18]:
(def render (comp ->infix simplify))

#'chp1/render

In [19]:
(render ((Lsphere 'm 'R) (up 't (up 'theta 'phi) (up 'thetadot 'phidot))))

"1/2 R² m phidot² sin²(theta) + 1/2 R² m thetadot²"

## The Metric

In [25]:
literal-metric

Syntax error compiling at (REPL:0:0).
Unable to resolve symbol: literal-metric in this context


class clojure.lang.Compiler$CompilerException: 

## Euler-Lagrange Residuals

In [26]:
(def gamma (literal-manifold-map 'q R1-rect R2-rect))

#'chp1/gamma

In [27]:
((point R1-rect) 't)

{:type :sicmutils.calculus.manifold/manifold-point, :spec t, :manifold {:manifold-family {:over Real, :name-format "R(%d)", :patch {:origin {:name :origin, :coordinate-system {:rectangular #function[sicmutils.calculus.manifold/->Rectangular], :polar-cylindrical #function[sicmutils.calculus.manifold/->PolarCylindrical]}}}}, :name "R(1)", :dimension 1, :embedding-dimension 1}, :coordinate-representation #atom[{#object[sicmutils.calculus.manifold$$reify__20397 0x147978b0 "sicmutils.calculus.manifold$$reify__20397@147978b0"] t} 0x46b29f0c]}

In [30]:
((chart R2-rect) (gamma ((point R1-rect) 't)))

(up (q↑0 t) (q↑1 t))

In [31]:
(def coordinate-path
    (compose (chart R2-rect) gamma (point R1-rect)))

#'chp1/coordinate-path

In [32]:
(coordinate-path 't)

(up (q↑0 t) (q↑1 t))